<a href="https://colab.research.google.com/github/FranciscoBPereira/AnaliseDados_2425_MEI_ISEC/blob/main/AD2425_P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 5)


# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

print('Python version: ', sys.version_info)
print('TF version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

**1. Data Fetching and Loading**





In [ ]:
# Download the Dataset and create a directory PetImages with two folders: Cat and Dog
# Each of the folders contains images from one class

!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

!unzip -q kagglecatsanddogs_5340.zip

!rm kagglecatsanddogs_5340.zip

In [ ]:
# Some images are corrupted, so they have to be removed

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images.")

In [ ]:
# Create datasets for training and validation

# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# https://www.tensorflow.org/guide/data

# The method image_dataset_from_directory() creates Dataset objects from images located in a specified directory
# https://keras.io/api/data_loading/image/

# Parameters subset and validation_split enable the creation of two datasets (Train: 80%; Validation: 20%)
# This method may shuffle images, adjust size and define the batch size
# This way the dataset is (almost) ready to be processed by the neural network

image_size = (128, 128)
batch_size = 32

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

class_names = train_ds.class_names


train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)


In [ ]:
# check the dimension of the created datasets

for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

print('Class Names: ', class_names)

In [ ]:
# Datasets are very important objects and they are associated with a large set of methods
# https://www.tensorflow.org/guide/data

for m in dir(tf.data.Dataset):
    if not (m.startswith("_") or m.endswith("_")):
        func = getattr(tf.data.Dataset, m)
        if hasattr(func, "__doc__"):
            print("● {:21s}{}".format(m + "()", func.__doc__.split("\n")[0]))

In [ ]:
# Visualize a few examples

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy()/255.0) # the image from the dataset is transformed into a numpy array
        plt.title(class_names[labels[i]])
        plt.axis("off")


**2. Creating and Training a Feed-forward Fully Connected Neural Network**

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
## Model_FF - FEED-FORWARD NN

# Create a feed-forward NN with Keras Sequential API: https://keras.io/api/models/

# Complete with the following architecture
# 2 hidden layers with 50 neurons each, He weight initialization and ReLU activation function
# Last hidden layer must be suitable for a classification problem with 2 classes

model_FF = keras.models.Sequential([
    layers.InputLayer(shape=(128,128,3)),
    keras.layers.Flatten(),
    keras.layers.Rescaling(1./255),

    ### COMPLETE ###
])

In [ ]:
# Summary

model_FF.summary()

In [ ]:
# Model compilation
# Define the loss function (https://keras.io/api/losses/)

loss_FF = ### COMPLETE ###

model_FF.compile(loss=loss_FF,
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

In [ ]:
# Train for 30 epochs

history = model_FF.fit(train_ds, epochs=30, validation_data=val_ds)

In [ ]:
# Results

import pandas as pd

x = pd.DataFrame(history.history, columns = ['accuracy', 'val_accuracy'])
x.plot(figsize=(8, 5))
plt.grid(True)
plt.show()

**Present**


1.   The last layer selected for your network
2.   The selected loss function
3.   A brief analysis of results


**3.	Creating a Simple CNN**

In [ ]:
# Model_CNN - Convolutional Neural Network

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Create a simple CNN

# Add the final classification layer to the model

model_CNN = keras.models.Sequential([
    layers.InputLayer(shape=(128,128,3)),
    layers.Rescaling(1./255),
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),

    ### COMPLETE ###

])


In [ ]:
model_CNN.summary()

**Questions:**

1.   How many weights has each kernel/filter of the second convolutional layer?
2.   How many feature maps are generated by the last convolutional layer?
3.   What is the dimension of each one of these feature maps?

In [ ]:
# Model compilation
# Define the loss function

lossCNN = ### COMPLETE ###

model_CNN.compile(loss= lossCNN,
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

history = model_CNN.fit(train_ds, epochs=30, validation_data=val_ds)

In [ ]:
# Plot the evolution of the accuracy metrics

import pandas as pd

x = pd.DataFrame(history.history, columns = ['accuracy', 'val_accuracy'])
x.plot(figsize=(8, 5))
plt.grid(True)
plt.show()

**Question:**

How do you analyse the results obtained by this CNN?

**4.Test the CNN with unseen images of your own pets**

1. Upload a jpg image to the working directory
2. Specify the name of the image
3. Apply the CNN and confirm the prediction

In [ ]:

Image_Name = '' #@param {type:"string"}

img = keras.utils.load_img(Image_Name, target_size=image_size)
plt.imshow(img)

img_array = keras.utils.img_to_array(img)
img_array = keras.ops.expand_dims(img_array, 0)  # Create batch axis

predictions = model_CNN.predict(img_array)
score = float(keras.ops.sigmoid(predictions[0][0]))
print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")

**5.	Implement and Test Modifications in the CNN**

In [ ]:


# Design and implement one change in the CNN and repeat the training process,
# seeking for an architecture that performs more effectively.

# Among other possibilities, you might consider one of the following points:
#  1. Change the CNN architecture, adding, deleting, or changing the parameterization of convolutional, maxpooling or dense layers.
#  2. Add Batch Normalization and/or Dropout layers.
#  3. Add a callback to implement Early Stopping.

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


# Create a new model called model_CNN2


###  CODE GOES HERE   ####

